In [1]:
import cv2
import numpy as np

# Load Yolo
net = cv2.dnn.readNet("custom-yolov4-tiny-detector_best.weights", "custom-yolov4-tiny-detector.cfg")
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Loading image
img = cv2.imread("parking-116_jpeg.rf.f949545b12bc6c447847c04467d50336.jpg")
# img = cv2.resize(img, None, fx=0.4, fy=0.4)
height, width, channels = img.shape

# Detecting objects
blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
net.setInput(blob)
outs = net.forward(output_layers)

# Showing informations on the screen
class_ids = []
confidences = []
boxes = []
for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            # Object detected
            center_x = int(detection[0] * width)
            center_y = int(detection[1] * height)
            w = int(detection[2] * width)
            h = int(detection[3] * height)
            # Rectangle coordinates
            x = int(center_x - w / 2)
            y = int(center_y - h / 2)
            boxes.append([x, y, w, h])
            confidences.append(float(confidence))
            class_ids.append(class_id)
            
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

xt,yt=2,1
img = cv2.resize(img, None, fx=xt, fy=yt)

font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
    if i in indexes:
        x, y, w, h = boxes[i]
        x*=xt
        y*=yt
        w*=xt
        h*=yt
        label = str(classes[class_ids[i]])
        if label=="unauthorized":
            label+=" afsd"
        color = colors[class_ids[i]]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
        cv2.putText(img, label, (x, y-2), font, 1, color, 2)
        
cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[ WARN:0@3.088] global /io/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('parking-116_jpeg.rf.f949545b12bc6c447847c04467d50336.jpg'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
import cv2
import numpy as np
import time
import random
from datetime import datetime

timer=time.time()

list=['EZ1I3D', 'K1JURX', 'XLETNF', 'OUDMST', 'PYFI0W', 'I8EXSQ', '9TCHGG', 'M2J8WC', 'NJGIUY', 'F2WVQA', 'YS161A', 'NPSR1K', 'POJWNS', 'VZTYCC', 'I1ZG44', 'RJRKPK', 'VV82DG', 'J5BR0V', 'OHX4H2', 'KUP2YG']

# Load Yolo
net = cv2.dnn.readNet("custom-yolov4-tiny-detector_best.weights", "custom-yolov4-tiny-detector.cfg")
classes = []
with open("obj.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))


video_capture = cv2.VideoCapture("2020-03-19-08-00-00_scene_0016_BB_write.avi")

if (video_capture.isOpened()== False):
    print("Error opening video stream or file")

frame_count=1
flag=0
while (video_capture.isOpened()):
    
    # Capture frame-by-frame
    ret, img = video_capture.read()
    if ret==True:
        
        frame_count+=1
        cv2.imshow('Video', img)
        height, width, channels = img.shape

        # Detecting objects
        blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
        net.setInput(blob)
        outs = net.forward(output_layers)

        # Showing informations on the screen
        class_ids = []
        confidences = []
        boxes = []
        for out in outs:
            for detection in out:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > 0.5:
                    # Object detected
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    # Rectangle coordinates
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)

        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.3)

        font = cv2.FONT_HERSHEY_PLAIN
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                if label=="unauthorized":
                    if flag==0:
                        now=datetime.now()
                        now=now.strftime("%d-%m-%Y %H:%M:%S")
                        print(now,random.choice(list))
                        flag=1
                    else:
                        label=label
                color = colors[class_ids[i]]
                cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
                cv2.putText(img, label, (x, y-2), font, 1, color, 2)
        cv2.imshow('Video', img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
        
timer2=time.time()
video_capture.release()
cv2.destroyAllWindows()
print("FPS: ",frame_count/(timer2-timer))